## Instalando as bibliotecas

In [ ]:
!pip install pycep-correios
!pip install nest_asyncio

## Lendo a tabela geolocation do HIVE

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import HiveContext

hive_context = HiveContext(sc)

spark = SparkSession \
    .builder \
    .appName("Teste API") \
    .enableHiveSupport() \
    .getOrCreate()

geo = spark.read.orc('/datalake/dadosbrutos/olist_geolocation_dataset.orc')

## Carregando os CEPs distintos em uma lista

In [ ]:
cep_array = [str(row.geolocation_zip_code_prefix) for row in geo.select('geolocation_zip_code_prefix').distinct().collect()]

## Consultando CEPs no site do Correios com apenas os 5 primeiros digitos
* Usando a API pycep conseguimos consultar apenas com o CEP completo, 8 digitos
* Consultando direto no site do Correios conseguimos consultar com 5 digitos e descobrir a cidade e estado do CEP

In [ ]:
import asyncio
import time
import aiohttp
import nest_asyncio
import pandas as pd

global results
results = []
global URL
# URL do site do correios
URL = 'https://buscacepinter.correios.com.br/app/endereco/carrega-cep-endereco.php'

# Função para pegar o primeiro resultado da pesquisa de CEP com apenas 5 digitos
async def get_address(session, cep):
    async with session.post(url=URL, data={'endereco': cep, 'tipoCEP': 'ALL'}) as response:
        response = await response.text()
        try:
            if json.loads(response)["dados"][0]["cep"] != '': 
                results.append(json.loads(response)["dados"][0])
                print(len(results), end="\r")
        except:
            pass

# Função para criar as tasks assíncronas, uma task para cada cep
async def get_all_addresses(ceps):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for cep in ceps:
            task = asyncio.ensure_future(get_address(session, cep))
            tasks.append(task)
        await asyncio.gather(*tasks, return_exceptions=False)
        

# Função prncipal para iniciar o loop assíncrono e criar o Dataframe com os resultados
if __name__ == "__main__":
    nest_asyncio.apply()
    start_time = time.time()
    ceps_array = cep_array
    asyncio.get_event_loop().run_until_complete(get_all_addresses(ceps_array))
    
    df = pd.DataFrame(results)
    df = df[['cep','localidade','uf']]
    
    duration = time.time() - start_time
    print(f"Downloaded {len(ceps_array)} ceps in {duration} seconds")

df.head(100)